### Function to scrape the data from krisha.kz  (Sale & Rent)

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_apartments_primary(rent=False, num_pages=10, delay=1):

    """
    Scrapes primary data about the apartment from Krisha.kz
    
    Args:
        rent (bool): Indicates which url to use. False for Sale apartment listng and True for Rent listing
        num_pages (int): Number of pages to scrape. Usually it is 1000 pages max to scrape.
        delay (int): How many seconds should be between requests. By default it's 1 second
    Returns:
        pd.DataFrame: DataFrame contains raw primary data about apartment
    """

    # Setting base url
    base_url = ""
    if rent:
        base_url = "https://krisha.kz/arenda/kvartiry/astana/"
    else:
        base_url = "https://krisha.kz/prodazha/kvartiry/astana/"
 
    apartments = []

    for page in range(1, num_pages + 1):
        url = f"{base_url}?page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        listings = soup.find_all("div", class_="a-card")

        for listing in listings:
            try:
                title = listing.find("a", class_="a-card__title").get_text(strip=True)
                price = listing.find("div", class_="a-card__price").get_text(strip=True)
                location = listing.find("div", class_="a-card__subtitle").get_text(strip=True)
                link = "https://krisha.kz" + listing.find("a", class_="a-card__title")['href']
                
                apartments.append([title, price, location, link])
            except AttributeError:
                continue
        
        # Respectful scraping delay
        time.sleep(delay)

    # Convert data to a DataFrame
    df = pd.DataFrame(apartments, columns=["Title", "Price", "Location", "Link"])
    
    return df

In [17]:
df = scrape_apartments_primary(rent=True, num_pages=3)

Collecting primary data for rent

In [20]:
num_pages = 422 # Maximum number of the pages for current time
df = scrape_apartments_primary(True, num_pages)
df

,Title,Price,Location,Link
0,1-комнатная квартира · 40 м² · помесячно,130 000〒,"Есильский р-н, Е-15 3",https://krisha.kz/a/show/760402765
1,3-комнатная квартира · 90 м² · 6/12 этаж · пом...,250 000〒,"Сарыарка р-н, Сарыарка 31/2",https://krisha.kz/a/show/698600297
2,2-комнатная квартира · 30 м² · 4/5 этаж · поме...,120 000〒,"Сарыарка р-н, Республика 76 — ЖД Вокзал старый",https://krisha.kz/a/show/698333334
3,1-комнатная квартира · 36 м² · 2/12 этаж · пом...,170 000〒,Чингиз Айтматов 60/1,https://krisha.kz/a/show/682941229
4,4-комнатная квартира · 160 м² · 18/26 этаж · п...,400 000〒,"Есильский р-н, Динмухаммед Кунаев",https://krisha.kz/a/show/761561653
...,...,...,...,...
8419,1-комнатная квартира · 38 м² · 8/10 этаж · пом...,190 000〒,"Есильский р-н, Култегин 13",https://krisha.kz/a/show/684109745
8420,2-комнатная квартира · 60 м² · 4/16 этаж · пом...,250 000〒,"Есильский р-н, Сыганак 15 — Е-305",https://krisha.kz/a/show/762270046
8421,1-комнатная квартира · 40 м² · 4/8 этаж · поме...,170 000〒,"Есильский р-н, Бухар жырау 40",https://krisha.kz/a/show/762269940
8422,2-комнатная квартира · 70 м² · 13/14 этаж · по...,300 000〒,"Есильский р-н, Алматы 13 — Алматы Мангилик ел",https://krisha.kz/a/show/683071225


Saving datasets

In [21]:
df.to_csv("astana_rent_raw_march_2025.csv")